In [10]:
#SCRAPING ENVOIREMENT

In [183]:
#Importing libaries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools
import random
import threading
from concurrent.futures import ThreadPoolExecutor

In [285]:
#Importing page-links
list_url = []
for page in range(1,7,1):
    url = f'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page={page}'
    list_url.append(url)

list_url





['https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=1',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=2',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=3',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=4',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=5',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=6']

In [287]:
#Retriving HTMLS
list_htmls =[]
for url in list_url:
    response = requests.get(url, headers={'name': 'Mathias Rasmussen', 'email': 'kbd655@alumni.ku.dk'})
    list_htmls.append(response.text)
    time.sleep(0.5)


    

In [289]:
# Finding links on appartments
links = []
for html in list_htmls:
    soup = BeautifulSoup(html, 'lxml')
    pattern = re.compile(r'href\":\"(/addresses/[^\"]+)')
    matches = pattern.findall(str(soup))

    apartments_list = list(set(matches))


    base_url = "https://www.boligsiden.dk"
    apartment_links = [base_url + link for link in apartments_list] #This list contains appartments
    links.append(apartment_links)

flattened_links = list(itertools.chain(*links))
print(len(flattened_links)) #Counting number of links

300


In [273]:
#Using selenium and beautifulsoup to retrive features + information from apartment-links 
#dict = {}
#links_med_fejl = []

for link in links_med_fejl:
    #response = requests.get(link, headers={'name': 'Mathias Rasmussen', 'email': 'kbd655@alumni.ku.dk'})
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(1,3))
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text()) #Searching for numbers in postal-code string.
        total_address = address[0].get_text()+','+pstnr_ny[0]
        #print(area_size)
        #driver.quit() #Closing driver
        try:
            if len(area_size)>2: #Bolig
                dict[total_address] = {'m2': area_size[0].get_text(), 'Grund':area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(), 'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0]}
            else: #Lejlighed
                dict[total_address] = {'m2': area_size[0].get_text(), 'Ejerudgift':area_size[1].get_text(), 'Elevator': other[0].get_text(), 'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilet': other[3].get_text(), 'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0]}
        except:
            dict[total_address] = {'m2': '?', 'Ejerudgift': '?', 'Elevator': '?', 'Rooms': '?', 'Build': '?', 'Toilet': '?', 'Altan': '?', 'Terasse': '?', 'Postal code': '?'}
    except:
        print('Her var der fejl i link:', link)
        links_med_fejl.append(link)
        continue
        #driver.quit()


In [291]:
#Carefull! Runs with 5 links at a time
dict = {}
links_med_fejl = []

def fetch_data(link):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'m2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0]}
            else:  # Lejlighed
                dict[total_address] = {'m2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilet': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0]}
        except:
            dict[total_address] = {'m2': '?', 'Ejerudgift': '?', 'Elevator': '?', 'Rooms': '?', 'Build': '?', 'Toilet': '?', 'Altan': '?', 'Terasse': '?', 'Postal code': '?'}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(fetch_data, flattened_links[0:300])


Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50ba-714d-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50b7-6fea-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50a7-183b-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50b2-0973-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50b8-d6f9-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c2-2203-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50ca-b5ed-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c8-b56e-32b8-e044-0003ba298018 - list index out of range
Error oc

In [123]:
#Printing features
print(dict)

for i in area_size:
    print(i.get_text())

for i in other:
    print(i.get_text())

print(total_address)



print(area_size[1].get_text())

{}
55 m²
Ejerudg.: 2.983 kr/md
Elevator: Nej
2 værelser
Opført 1939
1 toilet
Altan: Nej
Terrasse: Nej
Bremensgade 23, 2. tv.,2300
Ejerudg.: 2.983 kr/md


In [295]:
dataframe1 = pd.DataFrame.from_dict(dict, orient='index')

In [311]:
dataframe1

,m2,Grund,Ejerudgift,Rooms,Build,Toilets,Postal code,Elevator,Toilet,Altan,Terasse
"Feldborgvej 10,2770",96 m²,Grund: 236 m²,Ejerudg.: 4.094 kr/md,5 værelser,Opført 1951,1 toilet,2770,NaN,NaN,NaN,NaN
"Søndre Badevej 6C, 1. 3.,4600",141 m²,NaN,Ejerudg.: 6.153 kr/md,4 værelser,Opført 2018,NaN,4600,Elevator: Nej,2 toiletter,Altan: Nej,Terrasse: Nej
"Bakkedraget 14,4450",163 m²,Grund: 825 m²,Ejerudg.: 1.695 kr/md,5 værelser,Opført 1938,1 toilet,4450,NaN,NaN,NaN,NaN
"Thunøgade 50, st.,8000",50 m²,NaN,Ejerudg.: 2.701 kr/md,2 værelser,Opført 1895,NaN,8000,Elevator: Nej,1 toilet,Altan: Nej,Terrasse: Ja
"Parmagade 38, 5.,2300",70 m²,NaN,Ejerudg.: 3.582 kr/md,2 værelser,Opført 2020,NaN,2300,Elevator: Nej,1 toilet,Altan: Ja,Terrasse: Nej
...,...,...,...,...,...,...,...,...,...,...,...
"Engskovtoften 6,8541",165 m²,Grund: 763 m²,Ejerudg.: 3.024 kr/md,5 værelser,Opført 2005,2 toiletter,8541,NaN,NaN,NaN,NaN
"Bøstrupvej 1,4270",150 m²,Grund: 670 m²,Ejerudg.: 1.634 kr/md,4 værelser,Opført 1931,1 toilet,4270,NaN,NaN,NaN,NaN
"Sportsvej 20,3480",148 m²,Grund: 800 m²,Ejerudg.: 3.609 kr/md,5 værelser,Opført 1972,2 toiletter,3480,NaN,NaN,NaN,NaN
"Thorupvej 37,7451",270 m²,Grund: 6316 m²,Ejerudg.: 2.626 kr/md,6 værelser,Opført 1917,2 toiletter,7451,NaN,NaN,NaN,NaN
